<a href="https://colab.research.google.com/github/msaadsadiq/CFVAE/blob/master/CFVAE_ihdp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CFVAE - IHDP

###To do 
1. run on all ihdp and average
2. run ACIC dataset
3. start writing


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
# Set Hyper Parameters
EPS = 1e-15
z_red_dims = 16
gen_lr = 0.0001
reg_lr = 0.00001
b_size = 32
total_step = 100000

# writer = SummaryWriter()

In [3]:
# LEIE Data Load and normalize
csv_loader = []
df = pd.read_csv("/content/drive/My Drive/ihdp/csv/ihdp_npci_6.csv", header=None)
x= df.iloc[:, 5:].values
min_max = preprocessing.MinMaxScaler()
x_scaled = min_max.fit_transform(x)
dfx = pd.DataFrame(x_scaled)
dfx['trt'] = df.loc[:,0]
dfx['y'] = df.loc[:,1]
iter_per_epoch = int(dfx.shape[0]/b_size)
for i in range(iter_per_epoch):
    csv_loader.append(dfx.iloc[(i*b_size) : ((i+1)*b_size) , : ])

In [4]:
# Build Iterator 
data_iter = iter(csv_loader) #data_loader)    


def to_np(x):
    return x.data.cpu().numpy()

def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)    

In [5]:
#Encoder
class Q_net(nn.Module):  
    def __init__(self,X_dim,N,z_dim):
        super(Q_net, self).__init__()
        self.lin1 = nn.Linear(X_dim, N)
        self.lin2 = nn.Linear(N, N)
        self.lin3gauss = nn.Linear(N, z_dim)
    def forward(self, x):
        x = F.dropout(self.lin1(x), p=0.25, training=self.training)
        x = F.relu(x)
        x = F.dropout(self.lin2(x), p=0.25, training=self.training)
        x = F.relu(x)
        xgauss = self.lin3gauss(x)
        return xgauss

# Decoder
class P_net(nn.Module):  
    def __init__(self,X_dim,N,z_dim):
        super(P_net, self).__init__()
        self.lin1 = nn.Linear(z_dim, N)
        self.lin2 = nn.Linear(N, N)
        self.lin3 = nn.Linear(N, X_dim)
    def forward(self, x):
        x = F.dropout(self.lin1(x), p=0.2, training=self.training)
        x = F.relu(x)
        x = F.dropout(self.lin2(x), p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        return F.sigmoid(x)

      
class Net(nn.Module):
    def __init__(self, input_size=2, hidden_size=500):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)  
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        return self.fc4(out)

class Trt(nn.Module):
    def __init__(self, input_size=20, hidden_size=500, num_classes=1):
        super(Trt, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes) 
        # self.relu = nn.ReLU()
        # self.fc2 = nn.Linear(hidden_size, num_classes)    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        return F.sigmoid(out)


class FocalLoss(nn.Module):
    
    def __init__(self, weight=None, 
                 gamma=2., reduction='none'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob, 
            target_tensor, 
            weight=self.weight,
            reduction = self.reduction
        )

In [6]:
Q = Q_net(25,1000,z_red_dims).cuda()  #784
P = P_net(25,1000,z_red_dims).cuda()  #784
net_1 = Net(input_size = z_red_dims, hidden_size=500).cuda()
net_0 = Net(input_size = z_red_dims, hidden_size=500).cuda()
T = Trt(input_size = z_red_dims, hidden_size=100, num_classes=1).cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
loss_fn = torch.nn.MSELoss(reduction='sum')
FL = FocalLoss().cuda()


In [7]:
#encode/decode optimizers
optim_P = torch.optim.Adam(P.parameters(), lr=gen_lr)
optim_Q_enc = torch.optim.Adam(Q.parameters(), lr=gen_lr)
#regularizing optimizers
optim_Q_gen = torch.optim.Adam(Q.parameters(), lr=reg_lr)
optim_net1 = torch.optim.Adam(net_1.parameters(), lr=reg_lr)  
optim_net0 = torch.optim.Adam(net_0.parameters(), lr=reg_lr)  
optim_T = torch.optim.Adam(T.parameters(), lr=reg_lr)

In [8]:
# Start training
for step in range(total_step):

    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(csv_loader)

    # Fetch the images and labels and convert them to variables
    X, y = np.split(next(data_iter),[-1],axis=1) 
    X, trt = np.split(X, [-1], axis=1)
    X = torch.from_numpy(X.values).float()
    y = torch.from_numpy(y.values).float()
    trt = torch.from_numpy(trt.values).float()
    X, y, trt = to_var(X.view(X.size(0), -1)), to_var(y), to_var(trt)

    #reconstruction loss
    P.zero_grad()
    Q.zero_grad()
    T.zero_grad()
    net_1.zero_grad()
    net_0.zero_grad()
    
    z_sample = Q(X)   #encode to z
    X_sample = P(z_sample) #decode to X reconstruction
    recon_loss = F.binary_cross_entropy(X_sample+EPS,X+EPS) #loss_fn(X_sample,images)
    recon_loss.backward()

    optim_P.step()
    optim_Q_enc.step()
    
    # Discriminator
    Q.train()
    z_fake_gauss = Q(X)
    T_fake = T(z_fake_gauss)
    G_loss = F.binary_cross_entropy(T_fake+EPS,trt+EPS) #FL(argmax,trt).float().mean() 
    G_loss.backward()
    optim_Q_gen.step()
    

    # Q.eval()
    # z_fake_gauss = Q(X)
    # T_fake = T(z_fake_gauss)
    # T_loss = F.binary_cross_entropy(T_fake+EPS,trt+EPS) #FL(argmax,trt).float().mean() 
    # T_loss.backward()
    # optim_T.step()  

    # # for net 2
    # Q.eval()
    # net_2_real_gauss = net_2(z_real_gauss)
    # z_fake_gauss = Q(images)
    # net_2_fake_gauss = net_2(z_fake_gauss)
    # net_2_loss = loss_fn(net_2_real_gauss, net_2_fake_gauss) 
    # net_2_loss.backward()
    # optim_net2.step()
    # P.zero_grad()
    # Q.zero_grad()
    # net_1.zero_grad()
    # net_2.zero_grad()

    # Generator
    Q.eval()
    z_fake_gauss = Q(X)
    net_1_fake_gauss = net_1(z_fake_gauss)
    net_1_out_fake_gauss = trt * net_1_fake_gauss
    net1loss =  loss_fn(trt*y, net_1_out_fake_gauss)
    net1loss.backward()
    optim_net1.step()


    # for net 0, trt=0
    Q.eval()
    z_fake_gauss = Q(X)
    net_0_fake_gauss = net_0(z_fake_gauss)
    net_0_out_fake_gauss = (1-trt) * net_0_fake_gauss
    net0loss =  loss_fn( ((1-trt)*y), net_0_out_fake_gauss)
    net0loss.backward()
    optim_net0.step()

 

    # writer.add_scalar('Generator Loss', G_loss.data, step)
    # writer.add_scalar('Classification Loss', net_loss.data, step)
    # writer.add_scalar('Reconstruction Loss', recon_loss.data, step)
    
    if (step+1) % 100 == 0:
        print ('Step [%d/%d], recon_Loss: %.4f, G_Loss: %.4f, net1_Loss: %.4f, net2_Loss: %.4f, '
        %(step+1, total_step, recon_loss.data, G_loss.data, net1loss.data, net0loss.data))
                   

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Step [100/100000], recon_Loss: 0.4635, G_Loss: 0.6932, net1_Loss: 353.9930, net2_Loss: 114.8944, 
Step [200/100000], recon_Loss: 0.3529, G_Loss: 0.7154, net1_Loss: 81.2380, net2_Loss: 126.4741, 
Step [300/100000], recon_Loss: 0.3152, G_Loss: 0.6931, net1_Loss: 60.2674, net2_Loss: 46.5037, 
Step [400/100000], recon_Loss: 0.2240, G_Loss: 0.6909, net1_Loss: 11.5039, net2_Loss: 31.5900, 
Step [500/100000], recon_Loss: 0.1816, G_Loss: 0.6932, net1_Loss: 6.6861, net2_Loss: 57.1821, 
Step [600/100000], recon_Loss: 0.2330, G_Loss: 0.6931, net1_Loss: 9.6341, net2_Loss: 48.9395, 
Step [700/100000], recon_Loss: 0.2246, G_Loss: 0.7151, net1_Loss: 7.8090, net2_Loss: 53.1652, 
Step [800/100000], recon_Loss: 0.1674, G_Loss: 0.6928, net1_Loss: 17.9285, net2_Loss: 31.8213, 
Step [900/100000], recon_Loss: 0.1732, G_Loss: 0.7186, net1_Loss: 1.1977, net2_Loss: 51.9065, 
Step [1000/100000], recon_Loss: 0.1605, G_Loss: 0.7015, net1_Loss: 0.1459, net2_Loss: 22.9237, 
Step [1100/100000], recon_Loss: 0.1473, G

In [14]:
#save the Encoder
torch.save(Q.state_dict(),'Q_CF_IHDP_mask.pt')
torch.save(net_1.state_dict(),'Y1_ihdp_mask.pt')
torch.save(net_0.state_dict(),'Y0_ihdp_mask.pt')

# Evaluation

In [9]:
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from mpl_toolkits import mplot3d

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [18]:
def load_ihdp_data():
    csv_loader = []
    df = pd.read_csv("/content/drive/My Drive/ihdp/csv/ihdp_npci_7.csv", header=None)
    x= df.iloc[:, 5:].values
    min_max = preprocessing.MinMaxScaler()
    X = min_max.fit_transform(x)
    trt = df.iloc[:,0].to_numpy()
    y = df.iloc[:,1].to_numpy()
    ycf = df.iloc[:,2].to_numpy()
    return X,trt,y, ycf
    

def load_LEIE_data():
    df = pd.read_csv('LEIE_noNPI.csv', header= None)
    #create a balanced test set
    test = df.loc[ df[17] == 1 ]
    test = test.append( df.loc[ df[17] == 0 ].sample(n=5000, replace=False))   
    x= test.values
    min_max = preprocessing.MinMaxScaler()
    x_scaled = min_max.fit_transform(x)
    test = pd.DataFrame(x_scaled)
    images = test.iloc[:,:17].to_numpy()
    labels = test.iloc[:,17].to_numpy()
    return images, labels

def load_agg_data():
    df = pd.read_csv('agg_noNPI.csv', header= None)
    #create a balanced test set
    test = df.loc[ df[25] == True ]
    test = test.append( df.loc[ df[25] == False ].sample(n=5000, replace=False))   
    images = test.iloc[:,:25].to_numpy()
    labels = test.iloc[:,25].to_numpy()
    return images, labels

def load_NAgg_data():
    df = pd.read_csv('NA_FL_noNPI.csv', header= None)
    #create a balanced test set
    test = df.loc[ df[10] == 1 ]
    test = test.append( df.loc[ df[10] == 0 ].sample(n=5000, replace=False))
    x= test.values
    min_max = preprocessing.MinMaxScaler()
    x_scaled = min_max.fit_transform(x)
    test = pd.DataFrame(x_scaled)   
    images = test.iloc[:,:10].to_numpy()
    labels = test.iloc[:,10].to_numpy()
    return images, labels

def load_set_data():
    df = pd.read_csv('set_1.csv')
    images = df.iloc[:,4:24].to_numpy()
    labels = df.iloc[:,3].to_numpy()
    y = df.iloc[:,24].to_numpy()
    return images, labels, y

def load_trans_data():
    df = pd.read_csv('ee_embed_mean_cat.csv', header=None)
    del df[300]
    x= df.values
    min_max = preprocessing.MinMaxScaler()
    x_scaled = min_max.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    images = df.iloc[:,:300].to_numpy()
    labels = df.iloc[:,300].to_numpy()
    return images, labels

In [ ]:
def create_manifold():

    #Encoder
    class Q_net(nn.Module):  
        def __init__(self,X_dim,N,z_dim):
            super(Q_net, self).__init__()
            self.lin1 = nn.Linear(X_dim, N)
            self.lin2 = nn.Linear(N, N)
            self.lin3gauss = nn.Linear(N, z_dim)
        def forward(self, x):
            x = F.dropout(self.lin1(x), p=0.25, training=self.training)
            x = F.relu(x)
            x = F.dropout(self.lin2(x), p=0.25, training=self.training)
            x = F.relu(x)
            xgauss = self.lin3gauss(x)
            return xgauss                                        

    X, trt, y =  load_ihdp_data() 
    #load_NAgg_data() 
    #load_LEIE_data() 
    # #load_agg_data()

    # Convert to Tensors
    X = torch.from_numpy(X).float()
    trt = torch.from_numpy(trt).float()
    y = torch.from_numpy(y).float()

    X, trt, y = to_var(X.view(X.size(0), -1)), to_var(trt), to_var(y)

    # load the weights again
    z_red_dims = 12
    Q = Q_net(25,1000,z_red_dims).cuda()
    Q.load_state_dict(torch.load('Q_CF_IHDP_mask.pt'))
    Q.eval() #turrn off dropout

    # Run through encoder
    z_out = Q(X)

    # plot the manifold
    df_tsne = pd.DataFrame(z_out.data.cpu().numpy())

    Z_embed = TSNE(n_components=2, n_iter=1000, perplexity=50).fit_transform(df_tsne)
    df_tsne['trt'] = trt.data.cpu().numpy()

    df_tsne['tsne-2d-one'] = Z_embed[:,0]
    df_tsne['tsne-2d-two'] = Z_embed[:,1]
    
    plt.figure(figsize=(16,10))
    g = sns.scatterplot(
        x="tsne-2d-one", y="tsne-2d-two",
        hue="trt",
        # palette = sns.color_palette("Paired",3),
        #palette = sns.hls_palette(3, l=.3, s=.8),
        # palette=['pale red','medium green','denim blue'],
        palette=sns.color_palette("husl", 3),
        data=df_tsne,
        legend="full"#,
        #alpha=0.3
    )
    g.figure.savefig("z_ihdp_1.png")

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    xdata = Z_embed[:,0]
    ydata = Z_embed[:,1]
    zdata = Z_embed[:,2]

    ax.scatter3D(xdata, ydata, zdata, c=labels.data.cpu().numpy());
    fig.savefig('z_trans_sti_3d.png')

In [ ]:
def calc_acc():
    #Encoder
    class Q_net(nn.Module):  
        def __init__(self,X_dim,N,z_dim):
            super(Q_net, self).__init__()
            self.lin1 = nn.Linear(X_dim, N)
            self.lin2 = nn.Linear(N, N)
            self.lin3gauss = nn.Linear(N, z_dim)
        def forward(self, x):
            x = F.dropout(self.lin1(x), p=0.25, training=self.training)
            x = F.relu(x)
            x = F.dropout(self.lin2(x), p=0.25, training=self.training)
            x = F.relu(x)
            xgauss = self.lin3gauss(x)
            return xgauss  

    class Net(nn.Module):
        def __init__(self, input_size=2, hidden_size=500):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(input_size, hidden_size) 
            self.fc2 = nn.Linear(hidden_size, hidden_size)
            self.fc3 = nn.Linear(hidden_size, hidden_size)
            self.fc4 = nn.Linear(hidden_size, 1)  
        def forward(self, x):
            out = self.fc1(x)
            out = F.relu(out)
            out = self.fc2(out)
            out = F.relu(out)
            out = self.fc3(out)
            out = F.relu(out)
            return self.fc4(out)                                  

    X, trt, y, ycf = load_ihdp_data()

    # Convert to Tensors
    X = torch.from_numpy(X).float()
    trt = torch.from_numpy(trt).float()
    y = torch.from_numpy(y).float()
    ycf = torch.from_numpy(ycf).float()
    X, trt, y, ycf = to_var(X.view(X.size(0), -1)), to_var(trt), to_var(y), to_var(ycf)

    # load the weights again
    z_red_dims = 16
    Q = Q_net(25,1000,z_red_dims).cuda()
    Q.load_state_dict(torch.load('Q_CF_IHDP_mask.pt'))
    Q.eval() #turn off dropout

    net_1 = Net(input_size = z_red_dims, hidden_size=500).cuda()
    net_1.load_state_dict(torch.load('Y1_ihdp_mask.pt'))
    net_1.eval()
    net_0 = Net(input_size = z_red_dims, hidden_size=500).cuda()
    net_0.load_state_dict(torch.load('Y0_ihdp_mask.pt'))
    net_0.eval()

    z_out = Q(X)
    y1_hat = net_1(z_out)
    y0_hat = net_0(z_out)

    y11 = y[trt==1].data.cpu().numpy()
    y10 = ycf[trt==0].data.cpu().numpy()
    y00 = y[trt==0].data.cpu().numpy()
    y01 = ycf[trt==1].data.cpu().numpy()
    
    y1 = np.append(y11,y10)
    y0 = np.append(y01,y00)
    
    ATE = np.mean( np.mean(y1) - np.mean(y0)   )
    ATE_hat = y1_hat.data.cpu().numpy() - y0_hat.data.cpu().numpy()
    delta = abs( ATE - np.mean(ATE_hat))

    TE = np.mean(y1- y0)
    PEHE = np.sqrt(np.mean( (TE - ATE_hat)**2))

    ITE = np.mean(abs(TE - ATE_hat))

    # plot the manifold
    print("Eps_ITE Eps_ATE, PEHE,  :"ITE, delta, PEHE )

In [22]:
calc_acc()

ATE, PEHE, ITE : 0.28444242 1.739429 1.3612411
